In [19]:
import numpy as np
import cvxpy as cp

In [20]:
def water_filling(n, a, sum_x=1):
	'''
	Boyd and Vandenberghe, Convex Optimization, example 5.2 page 145
	Water-filling.

	This problem arises in information theory, in allocating power to a set of
	n communication channels in order to maximise the total channel capacity.
	The variable x_i represents the transmitter power allocated to the ith channel,
	and log(α_i+x_i) gives the capacity or maximum communication rate of the channel.
	The objective is to minimise -∑log(α_i+x_i) subject to the constraint ∑x_i = 1
	'''

	# Declare variables and parameters
	x = cp.Variable(shape=n)
	alpha = cp.Parameter(n, nonneg=True)
	alpha.value = a

	# Choose objective function. Interpret as maximising the total communication rate of all the channels
	obj = cp.Maximize(cp.sum(cp.log(alpha + x)))

	# Declare constraints
	constraints = [x >= 0, cp.sum(x) - sum_x == 0]

	# Solve
	prob = cp.Problem(obj, constraints)
	prob.solve()
	if(prob.status=='optimal'):
		return prob.status, prob.value, x.value
	else:
		return prob.status, np.nan, np.nan

In [21]:
buckets = 20
alpha = np.random.rand(buckets)


In [22]:
stat, prob, x = water_filling(buckets, alpha, buckets)
print('Problem status: {}'.format(stat))
print('Optimal communication rate = {:.4g} '.format(prob))
print('Transmitter powers:\n{}'.format(x))

Problem status: optimal
Optimal communication rate = 7.334 
Transmitter powers:
[0.96333793 1.33577852 0.45859963 0.6401124  1.41673003 1.06805109
 1.15386308 1.10012058 0.58716951 0.60130123 0.93345812 1.13755313
 1.30512893 1.23581804 1.36761027 1.03370868 0.75003383 0.47889851
 1.13109245 1.30163401]


In [23]:
ds = []
from tqdm import trange
for i in trange(10000):
	alpha = 1.5*np.random.rand(buckets) + 0.5
	
	stat, prob, x = water_filling(buckets, alpha, 1)
	if x is not np.nan:
		ds.append((alpha, x))

100%|██████████| 10000/10000 [00:30<00:00, 325.63it/s]


In [24]:
alpha = np.array([0.8, 1.0 ,1.2])
stat, prob, x = water_filling(buckets, alpha, 1)

ValueError: Invalid dimensions (3,) for Parameter value.

In [ ]:
x

array([0.53333977, 0.33333284, 0.13332738])

In [ ]:
ds[15][1]

array([1.36165943e-01, 8.63834049e-01, 7.04250557e-09])

In [ ]:
from pickle import dump
dump(ds, open("water_filling.pkl", "wb"))

In [ ]:
ds[-1]

(array([1.92031552, 1.42551773, 0.99026352]),
 array([7.26081215e-09, 2.82411140e-01, 7.17588851e-01]))